imports

In [1]:
import pandas as pd
import torch
import numpy as np
# import torchvision
import matplotlib.pyplot as plt
import time
import copy
from torch.utils.data import Dataset

In [2]:
# pd.read_csv("data_for_competition/test_target.csv")

In [3]:
file_names = [f'data_for_competition/train_data/train_data_{i}.pq' for i in range(12)]
# file_names

In [ ]:
s=0
for i in range(12):
    df = pd.read_parquet(f'data_for_competition/train_data/train_data_{i}.pq', engine='fastparquet')#[:100]
    s+=df.shape[0]
print(s)

In [8]:
df = pd.read_parquet(f'data_for_competition/train_data/train_data_{0}.pq', engine='fastparquet')[:100]

In [9]:
cat_columns = list(df.columns)[2:]


In [10]:
df[cat_columns] = df[cat_columns].astype('category')

In [15]:
df[:30]

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
5,0,6,5,0,11,8,12,11,4,2,...,3,3,3,4,1,2,3,1,0,1
6,0,7,3,9,1,2,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
7,0,8,2,9,2,3,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
8,0,9,1,9,11,13,14,8,2,5,...,3,3,3,4,1,2,4,1,0,0
9,0,10,7,9,2,10,8,8,16,4,...,3,3,3,4,1,2,4,1,0,0


In [ ]:
df.iloc[0]

In [ ]:
df

Loader

In [4]:
class CustomDataset(Dataset):
    def __init__(self, filenames, batch_size):
      # `filenames` is a list of strings that contains all file names.
      # `batch_size` determines the number of files that we want to read in a chunk.
        self.filenames = filenames
        self.batch_size = batch_size

    def __len__(self):
        # Number of chunks.
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))

    def __getitem__(self, idx):  # idx means index of the chunk.
      # In this method, we do all the preprocessing.
      # First read  data from files in a chunk. Preprocess it. Extract labels. Then return data and labels.
        # This extracts one batch of file names from the list `filenames`.
        batch_x = self.filenames[idx *
                                 self.batch_size:(idx + 1) * self.batch_size]
        data = []
        labels = []
        label_classes = ["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]
        for file in batch_x:
            # Change this line to read any other type of file
            temp = pd.read_parquet(file, engine='fastparquet' )
            # Convert column data to matrix like data with one channel
            data.append(temp.values)
        # Because of Pytorch's channel first convention
        data = np.asarray(data)
        labels = np.asarray(labels)

        # The following condition is actually needed in Pytorch. Otherwise, for our particular example, the iterator will be an infinite loop.
        # Readers can verify this by removing this condition.
        if idx == self.__len__():
            raise IndexError

        return data, labels


In [5]:
ds = CustomDataset(file_names,1)

In [6]:
for data, labels in ds:
    print(data.shape)
    break

(1, 1974724, 61)


In [7]:
data

array([[[     0,      1,     18, ...,      1,      0,      0],
        [     0,      2,     18, ...,      1,      0,      0],
        [     0,      3,     18, ...,      1,      1,      1],
        ...,
        [249999,      1,      9, ...,      1,      0,      1],
        [249999,      2,      9, ...,      1,      0,      0],
        [249999,      3,      9, ...,      1,      0,      0]]])

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model